Simple blend (equal weights)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ge = pd.read_csv('xgbm_submission_fi(xgb_166)_null0.csv')
jh = pd.read_csv('JH_submission(0.89).csv')
jg = pd.read_csv('Jaegeun_Catboost.csv')

In [4]:
ge.shape, jh.shape, jg.shape

((506691, 2), (506691, 2), (506691, 2))

In [5]:
ge.rename(columns={'isFraud':'isFraud_ge'}, inplace=True)
jh.rename(columns={'isFraud':'isFraud_jh'}, inplace=True)
jg.rename(columns={'isFraud':'isFraud_jg'}, inplace=True)

In [6]:
gh = pd.merge(ge, jh, left_on='TransactionID', right_on='TransactionID')
gh.head(2)

,TransactionID,isFraud_ge,isFraud_jh
0,3663549,0.0008,0.017992
1,3663550,0.0044,0.010498


In [8]:
ghg = pd.merge(gh, jg, left_on='TransactionID', right_on='TransactionID')
ghg.head(2)

,TransactionID,isFraud_ge,isFraud_jh,isFraud_jg
0,3663549,0.0008,0.017992,0.005351
1,3663550,0.0044,0.010498,0.040050


In [11]:
ghg['isFraud'] = (ghg.isFraud_ge+ghg.isFraud_jh+ghg.isFraud_jg)/3
ghg.head(2)

,TransactionID,isFraud_ge,isFraud_jh,isFraud_jg,isFraud
0,3663549,0.0008,0.017992,0.005351,0.008048
1,3663550,0.0044,0.010498,0.040050,0.018316


In [12]:
ghg.drop(['isFraud_ge','isFraud_jh','isFraud_jg'],axis = 1, inplace=True)

In [13]:
ghg.head(2)

,TransactionID,isFraud
0,3663549,0.008048
1,3663550,0.018316


In [14]:
submission_df = pd.DataFrame({'TransactionID':ghg.TransactionID,'isFraud':ghg.isFraud})

In [16]:
# 파일 제출
submission_df.reset_index(drop=True).to_csv('ghg_ensemble.csv', index=False)


**Private Score : 0.895434**<br>
**Public Score : 0.918889**

```
from sklearn.ensemble import VotingClassifier

voting = VotingClassifier(enumerate=[('rfc',gh.isFraud_jh),
                                    ('xgb',gh.isFraud_ge)], voting='soft', n_jobs=2)
voting = voting.fit(train, target)
prediction = voting.predict(test)
submission_df = pd.DataFrame({'TransactionID':gh.TransactionID,'isFraud':gh.isFraud})
```